In [1]:
import os
import pandas as pd
import numpy as np
import openai
import requests
import pandas as pd
import openai
import json

import logging
import time

from typing import List, Dict, Any, Optional
from enum import Enum
from dataclasses import dataclass

import openrouter



In [ ]:
# Import the openrouter module

# Store API key in a separate file
with open('../config/openrouter_api_key.txt', 'r') as f:
    api_key = f.read().strip()

openrouter.api_key = api_key

In [ ]:
# Store API key in a separate file
with open('../config/api_key.txt', 'r') as f:
    api_key = f.read().strip()

openai.api_key = api_key

In [4]:
disease = 'Parkinson'

In [ ]:
# Define file path
file_path_parquet = os.path.join('../data/raw', f'01_{disease.lower()}.parquet')

# Load the DataFrame from the Parquet file
df_covariates = pd.read_parquet(file_path_parquet)

# Display the DataFrame
print(df_covariates)

In [6]:
print(df_covariates.columns)

Index(['Index', 'NCTId', 'LeadSponsorClass', 'LeadSponsorName', 'Condition',
       'OfficialTitle', 'BriefTitle', 'Acronym', 'StudyType',
       'InterventionType', 'InterventionName', 'InterventionOtherName',
       'InterventionDescription', 'Phase', 'StudyFirstSubmitDate',
       'LastUpdateSubmitDate', 'CompletionDate', 'OverallStatus',
       'BriefSummary', 'IsFDARegulatedDevice', 'StartDate',
       'DetailedDescription', 'ConditionMeshTerm', 'PrimaryOutcomeDescription',
       'SecondaryOutcomeDescription', 'EnrollmentCount', 'EnrollmentType',
       'BaselineCategoryTitle', 'BaselinePopulationDescription',
       'BaselineTypeUnitsAnalyzed', 'OtherOutcomeDescription',
       'EligibilityCriteria', 'StudyPopulation', 'HealthyVolunteers',
       'ReferencePMID', 'LocationCountry', 'PrimaryOutcomeTimeFrame',
       'BaselineMeasureTitle', 'BaselineMeasureUnitOfMeasure',
       'BaselineMeasurementValue', 'GPT_summary'],
      dtype='object')


In [7]:
# Define the keywords to search for

keywords = ['tdcs', 't-dcs']

# Create a regex pattern for case-insensitive search
pattern = '|'.join(keywords)

# Filter the DataFrame where any column contains the pattern
filtered_df = df_covariates[df_covariates.apply(
    lambda row: row.astype(str).str.contains(pattern, case=False, na=False).any(), axis=1
)]

# Display the filtered DataFrame
# print(filtered_df)

print(f"Number of rows before filtering: {df_covariates.shape[0]}")
print(f"Number of rows after filtering: {filtered_df.shape[0]}")

Number of rows before filtering: 3985
Number of rows after filtering: 66

Number of rows after filtering: 66


removed one from the list after manual review

NCT03221413 - is a tacs 
NCT03217110 - is a tms


In [8]:
filtered_df = filtered_df[~filtered_df['NCTId'].isin(['NCT03217110', 'NCT03221413'])]
print(filtered_df)

     Index        NCTId LeadSponsorClass  \
150   None  NCT06145776            OTHER   
200   None  NCT04174573            OTHER   
232   None  NCT03191916            OTHER   
284   None  NCT02503930        OTHER_GOV   
305   None  NCT02315781            OTHER   
...    ...          ...              ...   
3852  None  NCT03638531            OTHER   
3861  None  NCT05752240            OTHER   
3906  None  NCT01113086            OTHER   
3931  None  NCT01100762            OTHER   
3948  None  NCT03227783            OTHER   

                                        LeadSponsorName  \
150                       Federal University of Paraíba   
200   Maharishi Markendeswar University (Deemed to b...   
232                                      Sanford Health   
284                    Tel-Aviv Sourasky Medical Center   
305                 Pacific Parkinson's Research Centre   
...                                                 ...   
3852                    University of Nevada, Las Vegas   

In [ ]:
os.makedirs('../data/filtered', exist_ok=True)
save_path = os.path.join('../data/filtered', '02_simple_filter.parquet')
filtered_df.to_parquet(save_path)

In [10]:
filtered_df['Index'] = range(1, len(filtered_df) + 1)
print(filtered_df)

      Index        NCTId LeadSponsorClass  \
150       1  NCT06145776            OTHER   
200       2  NCT04174573            OTHER   
232       3  NCT03191916            OTHER   
284       4  NCT02503930        OTHER_GOV   
305       5  NCT02315781            OTHER   
...     ...          ...              ...   
3852     60  NCT03638531            OTHER   
3861     61  NCT05752240            OTHER   
3906     62  NCT01113086            OTHER   
3931     63  NCT01100762            OTHER   
3948     64  NCT03227783            OTHER   

                                        LeadSponsorName  \
150                       Federal University of Paraíba   
200   Maharishi Markendeswar University (Deemed to b...   
232                                      Sanford Health   
284                    Tel-Aviv Sourasky Medical Center   
305                 Pacific Parkinson's Research Centre   
...                                                 ...   
3852                    University of Nevada, 

In [11]:
import pandas as pd

# Example: filtered_df has at least two columns: 
# Column A (some data), Column B (the NCT number), etc.

# Make a copy of your incoming DataFrame
df = filtered_df.copy()

# Suppose column B is named "NCT_ID"
# We create a new column (column C) called "ClinicalTrialsLink"
# Using Excel’s HYPERLINK formula ensures that the links are clickable in Excel.
df.insert(
    loc=2,  # Insert as the third column (which will appear as column C)
    column='ClinicalTrialsLink',
    value=df['NCTId'].apply(lambda x: f'=HYPERLINK("https://clinicaltrials.gov/study/{x}", "{x}")')
)




In [ ]:
# Save the filtered DataFrame to an Excel file
df.to_excel('../data/filtered/02_simple_filter.xlsx', index=False)

In [13]:
1=2

SyntaxError: cannot assign to literal here. Maybe you meant '==' instead of '='? (3293032332.py, line 1)